In [3]:
import requests
import time
import json

def get_studio_albums(artist_mbid, user_agent="MusicProject/1.0 (bittah.pupil@proton.me)"):
    """
    Fetch all studio album releases for an artist from MusicBrainz.
    
    Args:
        artist_mbid: MusicBrainz ID for the artist
        user_agent: User agent string for API requests
    
    Returns:
        List of dictionaries containing album information
    """
    base_url = "https://musicbrainz.org/ws/2"
    headers = {
        "User-Agent": user_agent,
        "Accept": "application/json"
    }
    
    studio_albums = []
    offset = 0
    limit = 100
    
    while True:
        # Query for release groups (albums) by the artist
        url = f"{base_url}/release-group"
        params = {
            "artist": artist_mbid,
            "type": "album",  # Only albums, not singles/EPs
            "limit": limit,
            "offset": offset,
            "fmt": "json"
        }
        
        response = requests.get(url, headers=headers, params=params)
        
        if response.status_code != 200:
            print(f"Error: {response.status_code}")
            break
        
        data = response.json()
        release_groups = data.get("release-groups", [])
        
        if not release_groups:
            break
        
        for rg in release_groups:
            # Filter out live albums and compilations
            secondary_types = rg.get("secondary-types", [])
            
            if "Live" not in secondary_types and "Compilation" not in secondary_types:
                album_info = {
                    "title": rg.get("title"),
                    "mbid": rg.get("id"),
                    "first_release_date": rg.get("first-release-date", "Unknown"),
                    "primary_type": rg.get("primary-type"),
                    "secondary_types": secondary_types
                }
                studio_albums.append(album_info)
        
        # Check if there are more results
        if len(release_groups) < limit:
            break
        
        offset += limit
        time.sleep(1)  # Rate limiting - be nice to the API
    
    return studio_albums


# Example usage
if __name__ == "__main__":
    # Example: Radiohead's MBID
    artist_mbid = "a74b1b7f-71a5-4011-9441-d0b5e4122711"
    
    print(f"Fetching studio albums for artist MBID: {artist_mbid}\n")
    albums = get_studio_albums(artist_mbid)
    
    print(f"Found {len(albums)} studio albums:\n")
    for album in sorted(albums, key=lambda x: x["first_release_date"]):
        print(f"- {album['title']} ({album['first_release_date']})")

Fetching studio albums for artist MBID: a74b1b7f-71a5-4011-9441-d0b5e4122711

Found 15 studio albums:

- On a Friday Demos II ()
- On a Friday Demos 1 ()
- Pablo Honey (1993-02-22)
- The Bends (1994-11-29)
- OK Computer (1997-05-21)
- An Interview with Jonny and Colin Greenwood (2000)
- Kid A (2000-08-03)
- Remix Project (2001)
- Amnesiac (2001-06-04)
- Hail to the Thief (2003-05-26)
- Greenhouse Effect vs. Radiohead (2005-08)
- In Rainbows (2007-10-10)
- The Interview (2010-08-23)
- The King of Limbs (2011-02-18)
- A Moon Shaped Pool (2016-05-08)
